Para poder integrar la forma de pago al cliente, hay que unir las tablas pago con poliza 

In [1]:
import pandas as pd
import numpy as np

In [11]:
#df_pago = pd.read_table("../../data/raw/TB_PAGOS_CL.tsv", encoding="latin-1", low_memory=False, index_col=False)
#df_pago.info()

df_pago = pd.read_csv("../../data/processed/interm-BD_INT_PAGO_POLIZA_demora_pago.completo.csv", low_memory=False, index_col="Unnamed: 0")
#df = df.iloc[:,1:]
df_pago.info()



/home/seba/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4109394 entries, 0 to 4859237
Data columns (total 15 columns):
POLICY_NUMBER      int64
DUE_DATE           object
PAYMENT_DATE       object
PAYMENT_AMOUNT     object
MODE_OF_PAYMENT    object
PAYMENT_STATUS     object
CUSTOMER_ID        int64
fecha              object
next_f             object
mes                int64
anio               int64
corresp_date       object
demora             int64
pago_comp          int64
pago_deuda         int64
dtypes: int64(7), object(8)
memory usage: 501.6+ MB


In [12]:
df_pago.head()

,POLICY_NUMBER,DUE_DATE,PAYMENT_DATE,PAYMENT_AMOUNT,MODE_OF_PAYMENT,PAYMENT_STATUS,CUSTOMER_ID,fecha,next_f,mes,anio,corresp_date,demora,pago_comp,pago_deuda
0,77320269999,2004-01,2004-01,420,PP,CT,5519768,2004-01-01,2004-02-01,2,2004,2004-02,0,1,0
42,79741709999,2004-01,2004-01,1088,PP,CT,1406154,2004-01-01,2004-02-01,2,2004,2004-02,0,1,0
98,86520099999,2004-01,2004-01,660,PP,CT,3191732,2004-01-01,2004-02-01,2,2004,2004-02,0,1,0
188,92068329999,2004-01,2004-01,1776,PP,CT,1406154,2004-01-01,2004-02-01,2,2004,2004-02,0,1,0
320,100004239999,2004-01,2004-01,405,PP,CT,5926562,2004-01-01,2004-02-01,2,2004,2004-02,0,1,0


In [20]:
#agrego la cantidad de pagos para luego determinar el modo de pago
df_mop = pd.DataFrame(df_pago.groupby("CUSTOMER_ID")["MODE_OF_PAYMENT"].nunique())
df_mop.head()

,MODE_OF_PAYMENT
CUSTOMER_ID,
1690,1
5575,3
7968,1
9028,2
9860,3


In [21]:
df_mop.reset_index(level=0, inplace=True)
df_mop.head()

,CUSTOMER_ID,MODE_OF_PAYMENT
0,1690,1
1,5575,3
2,7968,1
3,9028,2
4,9860,3


In [23]:
df_mop["CUSTOMER_ID"] = df_mop.CUSTOMER_ID.astype(int)
df_mop.head()

,CUSTOMER_ID,MODE_OF_PAYMENT
0,1690,1
1,5575,3
2,7968,1
3,9028,2
4,9860,3


In [24]:
#df_con[df_mop[df_mop["MODE_OF_PAYMENT"] == 1].CUSTOMER_ID.values]

df_con_mop = pd.merge(df_pago, df_mop, how="left", suffixes=('_1', '_2') , on='CUSTOMER_ID'  )
df_con_mop.head()

,POLICY_NUMBER,DUE_DATE,PAYMENT_DATE,PAYMENT_AMOUNT,MODE_OF_PAYMENT_1,PAYMENT_STATUS,CUSTOMER_ID,fecha,next_f,mes,anio,corresp_date,demora,pago_comp,pago_deuda,MODE_OF_PAYMENT_2
0,77320269999,2004-01,2004-01,420,PP,CT,5519768,2004-01-01,2004-02-01,2,2004,2004-02,0,1,0,4
1,79741709999,2004-01,2004-01,1088,PP,CT,1406154,2004-01-01,2004-02-01,2,2004,2004-02,0,1,0,4
2,86520099999,2004-01,2004-01,660,PP,CT,3191732,2004-01-01,2004-02-01,2,2004,2004-02,0,1,0,3
3,92068329999,2004-01,2004-01,1776,PP,CT,1406154,2004-01-01,2004-02-01,2,2004,2004-02,0,1,0,4
4,100004239999,2004-01,2004-01,405,PP,CT,5926562,2004-01-01,2004-02-01,2,2004,2004-02,0,1,0,2


In [25]:
df_con_mop.drop_duplicates(inplace=True)

In [26]:
df_con_mop["ModoPago"] = np.nan
df_con_mop.loc[df_con_mop["MODE_OF_PAYMENT_2"] >1, "ModoPago"] = "Mix"
df_con_mop.loc[df_con_mop["MODE_OF_PAYMENT_2"] == 1, "ModoPago"] = df_con_mop[df_con_mop["MODE_OF_PAYMENT_2"] == 1]["MODE_OF_PAYMENT_1"]
df_con_mop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4109394 entries, 0 to 4109393
Data columns (total 17 columns):
POLICY_NUMBER        int64
DUE_DATE             object
PAYMENT_DATE         object
PAYMENT_AMOUNT       object
MODE_OF_PAYMENT_1    object
PAYMENT_STATUS       object
CUSTOMER_ID          int64
fecha                object
next_f               object
mes                  int64
anio                 int64
corresp_date         object
demora               int64
pago_comp            int64
pago_deuda           int64
MODE_OF_PAYMENT_2    int64
ModoPago             object
dtypes: int64(8), object(9)
memory usage: 564.3+ MB


In [27]:
df_con_mop["ModoPago"].value_counts()

Mix    2217148
CC      875975
TM      587648
PP      285617
BA      133856
TA        7883
SJ        1267
Name: ModoPago, dtype: int64

In [30]:
# cargo bd de datos personales para integrar todo a una nueva bd
df_cliente = pd.read_table("../../data/raw/TB_CLIENTE_FECHA.tsv", encoding="latin-1")
df_cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
CUSTOMER_ID        50000 non-null int64
DATE_OF_BIRTH      48937 non-null object
GENDER             49975 non-null object
MARITAL_STATUS     45029 non-null object
EDUCATION          16129 non-null object
EDUCATION_LEVEL    1409 non-null object
OCCUPATION         28792 non-null object
CITY               38608 non-null object
LOCALITY           38608 non-null object
FECHA_PROCESO      50000 non-null object
dtypes: int64(1), object(9)
memory usage: 3.8+ MB


In [31]:
df_cliente["DATE_OF_BIRTH"] = pd.to_datetime(df_cliente.DATE_OF_BIRTH)
df_cliente.info()
from datetime import datetime

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)

hoy = datetime.strptime("2018-01-01", "%Y-%m-%d")
df_cliente["edad"] = df_cliente.DATE_OF_BIRTH.apply(lambda x : ((hoy - x).days // 365) )
df_cliente.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
CUSTOMER_ID        50000 non-null int64
DATE_OF_BIRTH      48937 non-null datetime64[ns]
GENDER             49975 non-null object
MARITAL_STATUS     45029 non-null object
EDUCATION          16129 non-null object
EDUCATION_LEVEL    1409 non-null object
OCCUPATION         28792 non-null object
CITY               38608 non-null object
LOCALITY           38608 non-null object
FECHA_PROCESO      50000 non-null object
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 3.8+ MB


,CUSTOMER_ID,DATE_OF_BIRTH,GENDER,MARITAL_STATUS,EDUCATION,EDUCATION_LEVEL,OCCUPATION,CITY,LOCALITY,FECHA_PROCESO,edad
0,10374168,1953-10-31,F,Divorciado,N,NaN,ADMI,BUENOS AIRES,ITUZAINGO,31/01/2014,64.0
1,12897415,1956-12-02,M,Casado,NaN,NaN,NaN,SANTA FE,AVELLANEDA,31/01/2014,61.0
2,1386644,1951-12-24,M,Casado,NaN,NaN,NaN,NaN,NaN,31/01/2014,66.0
3,6118007,1950-08-13,F,Casado,NaN,NaN,EMPL,NaN,NaN,31/01/2014,67.0
4,4714574,1967-08-30,F,Casado,NaN,NaN,DOCE,BUENOS AIRES,SAN MARTIN,31/01/2014,50.0


In [32]:
df_c_mop = pd.merge(df_cliente, df_con_mop[["CUSTOMER_ID", "ModoPago"]], how="left", suffixes=('_cli', '_mop') , on='CUSTOMER_ID'  )
df_c_mop.head()

,CUSTOMER_ID,DATE_OF_BIRTH,GENDER,MARITAL_STATUS,EDUCATION,EDUCATION_LEVEL,OCCUPATION,CITY,LOCALITY,FECHA_PROCESO,edad,ModoPago
0,10374168,1953-10-31,F,Divorciado,N,NaN,ADMI,BUENOS AIRES,ITUZAINGO,31/01/2014,64.0,Mix
1,10374168,1953-10-31,F,Divorciado,N,NaN,ADMI,BUENOS AIRES,ITUZAINGO,31/01/2014,64.0,Mix
2,10374168,1953-10-31,F,Divorciado,N,NaN,ADMI,BUENOS AIRES,ITUZAINGO,31/01/2014,64.0,Mix
3,10374168,1953-10-31,F,Divorciado,N,NaN,ADMI,BUENOS AIRES,ITUZAINGO,31/01/2014,64.0,Mix
4,10374168,1953-10-31,F,Divorciado,N,NaN,ADMI,BUENOS AIRES,ITUZAINGO,31/01/2014,64.0,Mix


In [33]:
df_c_mop.drop_duplicates(inplace=True)
df_c_mop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 4109334
Data columns (total 12 columns):
CUSTOMER_ID        50000 non-null int64
DATE_OF_BIRTH      48937 non-null datetime64[ns]
GENDER             49975 non-null object
MARITAL_STATUS     45029 non-null object
EDUCATION          16129 non-null object
EDUCATION_LEVEL    1409 non-null object
OCCUPATION         28792 non-null object
CITY               38608 non-null object
LOCALITY           38608 non-null object
FECHA_PROCESO      50000 non-null object
edad               48937 non-null float64
ModoPago           50000 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(9)
memory usage: 5.0+ MB


In [34]:
df_c_mop.ModoPago.value_counts()

Mix    23637
PP      8796
CC      8147
TM      7485
BA      1876
TA        49
SJ        10
Name: ModoPago, dtype: int64

In [37]:
#hacer bins para la edad
bins = [-9999, 0, 18, 30, 45, 60, 85, 99999]
labels = [0, 1, 2, 3, 4, 5, -1 ]
df_c_mop["edad_r"] = pd.cut(df_c_mop.edad, bins=bins, labels=labels)
df_c_mop.loc[df_c_mop["edad_r"] == -1, "edad_r"] = 0

In [39]:
df_c_mop["edad_r"].value_counts()

 3    15379
 4    15071
 5    11841
 2     3489
 0     2851
 1      306
-1        0
Name: edad_r, dtype: int64

In [40]:
df_c_mop.to_csv("../../data/processed/BD_ID_CLIENTE_D.Pers_Modo.Pago.csv")

In [41]:
df_c_mop.head()

,CUSTOMER_ID,DATE_OF_BIRTH,GENDER,MARITAL_STATUS,EDUCATION,EDUCATION_LEVEL,OCCUPATION,CITY,LOCALITY,FECHA_PROCESO,edad,ModoPago,edad_r
0,10374168,1953-10-31,F,Divorciado,N,NaN,ADMI,BUENOS AIRES,ITUZAINGO,31/01/2014,64.0,Mix,5
86,12897415,1956-12-02,M,Casado,NaN,NaN,NaN,SANTA FE,AVELLANEDA,31/01/2014,61.0,Mix,5
112,1386644,1951-12-24,M,Casado,NaN,NaN,NaN,NaN,NaN,31/01/2014,66.0,Mix,5
181,6118007,1950-08-13,F,Casado,NaN,NaN,EMPL,NaN,NaN,31/01/2014,67.0,Mix,5
300,4714574,1967-08-30,F,Casado,NaN,NaN,DOCE,BUENOS AIRES,SAN MARTIN,31/01/2014,50.0,Mix,4
